In [6]:
from kafka import KafkaConsumer
from pymongo import MongoClient
from json import loads
import logging

# Logger configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Kafka and MongoDB configurations
KAFKA_TOPIC = 'vnexpress'
KAFKA_SERVER = 'localhost:9092'
MONGO_URI = 'mongodb://localhost:27017'
MONGO_DB = 'vnexpress'
MONGO_COLLECTION = 'vnexpress'

def main():
    try:
        # Initialize Kafka Consumer
        consumer = KafkaConsumer(
            KAFKA_TOPIC,
            bootstrap_servers=[KAFKA_SERVER],
            auto_offset_reset='earliest',  # Start reading from the earliest offset
            enable_auto_commit=True,  # Automatically commit offsets
            group_id='my-group',
            value_deserializer=lambda x: loads(x.decode('utf-8'))
        )
        logging.info(f"Connected to Kafka topic: {KAFKA_TOPIC}")

        # Initialize MongoDB client
        client = MongoClient(MONGO_URI)
        db = client[MONGO_DB]
        collection = db[MONGO_COLLECTION]
        logging.info(f"Connected to MongoDB collection: {MONGO_DB}.{MONGO_COLLECTION}")

        # Process messages from Kafka
        for message in consumer:
            data = message.value
            try:
                # Insert message into MongoDB
                collection.insert_one(data)
                logging.info(f"Message inserted into MongoDB: {data}")
            except Exception as e:
                logging.error(f"Error inserting into MongoDB: {e}")
    except Exception as e:
        logging.error(f"Error connecting to Kafka or MongoDB: {e}")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("Stopped")  # Print 'break' when Ctrl+C is pressed


2024-12-30 15:31:51,108 - INFO - <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
2024-12-30 15:31:51,111 - INFO - Probing node bootstrap-0 broker version
2024-12-30 15:31:51,113 - INFO - <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: Connection complete.
2024-12-30 15:31:51,222 - INFO - Broker version identified as 2.5.0
2024-12-30 15:31:51,224 - INFO - Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
2024-12-30 15:31:51,226 - INFO - Updating subscribed topics to: ('vnexpress',)
2024-12-30 15:31:51,227 - INFO - Connected to Kafka topic: vnexpress
2024-12-30 15:31:51,240 - INFO - Connected to MongoDB collection: vnexpress.vnexpress
2024-12-30 15:31:51,248 - INFO - Group coordinator for my-group is BrokerMetadata(nodeId='coordinator-0', host='LamP', port=9092, rack=None)
2024-12-30 15:31:51,2

Stopped
